In [8]:
import pandas as pd
import csv
import json
import aisuite as ai
import re
client = ai.Client()

In [9]:
with open('../data/questions.json', 'r') as f:
    questions_data = json.load(f)
    questions = [entry["binary"] for entry in questions_data[10:13]] #working with 3 questions for now
    statements = [entry["likert"] for entry in questions_data[10:13]]

# load the full summary dataframe
df = pd.read_csv('../data/movie_book_dataframe.csv')

# extract synopsis to run a test
test_summaries = df[df['Title'].isin(['Snowpiercer', 'Dracula'])]

# select models
models = ["openai:gpt-4o-mini"] #"anthropic:claude-3-5-sonnet-20240620"

In [10]:
all_prompts = [
    {
        "title": row['Title'],
        "summary": row['Summary'],
        "binary": 
        { "messages": [
            {"role": "system", "content": 
             "You are a story analyzer. Your task is to analyze synopses regarding a specific question. Your answers have this structure <Reasoning></Reasoning> <Final response>yes/no</Final response>"}, 
            {"role": "user", "content": f"<Synopsis> {row['Summary']} </Synopsis> <Question>Answer the following question: {question}</Question>"}
        ]} ,
        "likert": {"messages": [
            {"role": "system", "content": 
             "You are a story analyzer. Your task is to analyze synopses regarding a given hypothesis. You indicate your agreement on a scale from 1 (strongly disagree) to 7 (strongly agree). Your answers have this structure <Reasoning></Reasoning> <Final response>1-7</Final response>"}, 
            {"role": "user", "content": f"<Synopsis> {row['Summary']} </Synopsis> <Question>How much do you agree with the following statement (1-7): {statement}</Question>"}
        ]}
    }
    for _, row in test_summaries.iterrows()
    for question, statement in zip(questions, statements)
]

In [12]:
for model in models:
    for i in range(len(all_prompts)):
        question_prompt = all_prompts[i]["binary"]["messages"]
        statement_prompt = all_prompts[i]["likert"]["messages"]

        binary_response = client.chat.completions.create(
        model=model,
        messages=question_prompt,
        temperature=0)

        likert_response = client.chat.completions.create(
        model=model,
        messages=statement_prompt,
        temperature=0)

        all_prompts[i]["binary"][model] = binary_response.choices[0].message.content
        all_prompts[i]["likert"][model] = likert_response.choices[0].message.content

        binary_match = re.search(r"<Final response>(.*?)</Final response>", all_prompts[i]["binary"][model])
        all_prompts[i]["binary"][f"final_{model}"] = binary_match.group(1).strip() if binary_match else None

        # Extract likert response
        likert_match = re.search(r"<Final response>(\d+)</Final response>", all_prompts[i]["likert"][model])
        all_prompts[i]["likert"][f"final_{model}"] = int(likert_match.group(1)) if likert_match else None

        #update json file
        with open('../data/ai_responses.json', 'w') as f:
            json.dump(all_prompts, f, indent=4)